# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import time

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

604

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
# Establishing url and query url for API calls
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = url + "q=" + city + "&APPID=" + weather_api_key

# Set record count and set count so that we have a starting point
# for counting records in groups (sets) of 50
record_count = 1
set_count = 1

# Print header for log
print("Beginning Data Retrieval")
print("------------------------")

# Number the cities list and loop through until record count reaches
# 50, then restart record count. 
for i, city in enumerate(cities):
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # Make API call and print call information
    query_url
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add one to record count before looping back
    record_count += 1
    
    # Set timer so as not to make more requests per minute than allotted by API
    time.sleep(1)

Beginning Data Retrieval
------------------------
Processing Record 1 of Set 1 | yashkul
Processing Record 2 of Set 1 | medea
Processing Record 3 of Set 1 | nikolskoye
Processing Record 4 of Set 1 | doha
Processing Record 5 of Set 1 | santa marinella
Processing Record 6 of Set 1 | saint-philippe
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | tual
Processing Record 9 of Set 1 | punta arenas
Processing Record 10 of Set 1 | dikson
Processing Record 11 of Set 1 | lebu
Processing Record 12 of Set 1 | gat
Processing Record 13 of Set 1 | christchurch
Processing Record 14 of Set 1 | lashio
Processing Record 15 of Set 1 | tuatapere
Processing Record 16 of Set 1 | bengkulu
Processing Record 17 of Set 1 | east london
Processing Record 18 of Set 1 | puerto ayora
Processing Record 19 of Set 1 | belushya guba
Processing Record 20 of Set 1 | jamestown
Processing Record 21 of Set 1 | busselton
Processing Record 22 of Set 1 | bredasdorp
Processing Record 23 of Set 1 | torbay
Pro

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [16]:
# Set empty lists for creating dataframe
name = []
lat = []
lng = []
max_temp = []
humidity = []
cloud = []
wind = []
country = []
date = []

# Loop through city names and get weather data for each one
for city in cities:
    try:
        weather_response = requests.get(url + "q=" + city + "&APPID=" + weather_api_key).json()
        
        name.append(weather_response['name'])
        lat.append(weather_response['coord']['lat'])
        lng.append(weather_response['coord']['lon'])
        max_temp.append(weather_response['main']['temp_max'])
        humidity.append(weather_response['main']['humidity'])
        cloud.append(weather_response['clouds']['all'])
        wind.append(weather_response['wind']['speed'])
        country.append(weather_response['sys']['country'])
        date.append(weather_response['dt'])
    
    # Skip city entries with no data
    except (KeyError, IndexError):
        pass    
    
    # Set timer so as not to make more requests per minute than allotted by API
    time.sleep(1)

# Create dictionary to convert to dataframe
cities_weather_dict = {
    "City": name,
    "Lat": lat,
    "Lng": lng,
    "Max Temp": max_temp,
    "Humidity": humidity,
    "Cloudiness": cloud,
    "Wind Speed": wind,
    "Country": country,
    "Date": date
}

# Create dataframe from dict
cities_weather = pd.DataFrame(cities_weather_dict)

# Display weather dataframe
cities_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yashkul',46.1711,45.3435,280.56,73,100,2.92,RU,1612391017
1,Médéa,36.2642,2.7539,284.75,48,100,3.46,DZ,1612390741
2,Nikolskoye,59.7035,30.7861,267.04,92,90,8.00,RU,1612391019
3,Doha,25.2867,51.5333,291.15,72,0,1.03,QA,1612390944
4,Santa Marinella,42.0345,11.8536,283.15,92,57,7.60,IT,1612391021


In [17]:
cities_weather.count()

City          556
Lat           556
Lng           556
Max Temp      556
Humidity      556
Cloudiness    556
Wind Speed    556
Country       556
Date          556
dtype: int64

In [18]:
cities_weather.to_csv('cities_weather.csv')

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [4]:
#  Get the indices of cities that have humidity over 100%.


In [5]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression